<a href="https://colab.research.google.com/github/aditya-shriwastava/DeepRL_Experimentation/blob/master/PolicyGradient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup the project in Google Colab

In [1]:
#@title Mount the project in Google Colab

GUEST = False
PROJECT = 'DeepRL_Experimentation'

import os
if GUEST:
  !git clone https://github.com/aditya-shriwastava/DeepRL_Experimentation.git
else:
  # Mount Google Drive
  from google.colab import drive
  drive.mount('/content/gdrive')

  # Setup Symlink to project folder
  DRIVE_PATH = '/content/gdrive/My\ Drive/' + PROJECT
  SYM_PATH = '/content/' + PROJECT
  if not os.path.exists(SYM_PATH):
    !ln -s $DRIVE_PATH $SYM_PATH

os.chdir( '/content/' + PROJECT)

Mounted at /content/gdrive


In [2]:
#@title apt install requirements

!apt update > /dev/null 2>&1
!apt install -y --no-install-recommends \
        build-essential \
        curl \
        git \
        gnupg2 \
        make \
        cmake \
        ffmpeg \
        swig \
        libz-dev \
        unzip \
        zlib1g-dev \
        libglfw3 \
        libglfw3-dev \
        libxrandr2 \
        libxinerama-dev \
        libxi6 \
        libxcursor-dev \
        libgl1-mesa-dev \
        libgl1-mesa-glx \
        libglew-dev \
        libosmesa6-dev \
        lsb-release \
        ack-grep \
        patchelf \
        wget \
        xpra \
        xserver-xorg-dev \
        xvfb \
        python-opengl \
        ffmpeg > /dev/null 2>&1

In [3]:
#@title pip install requirements

!pip install pyvirtualdisplay > /dev/null 2>&1
!pip3 install box2d-py > /dev/null 2>&1
!pip install Box2D > /dev/null 2>&1

In [4]:
#@title Setup Virtual Display

from pyvirtualdisplay import Display

display = Display(visible=0, size=(1400, 900))
display.start()

# Test if setup is completed and working properly

In [5]:
import gym
from colab_utils import (
    wrap_env,
    show_video
)

env = wrap_env(gym.make("CartPole-v1"))

observation = env.reset()
for i in range(100):
    env.render(mode='rgb_array')
    obs, rew, term, _ = env.step(env.action_space.sample() ) 
    if term:
      break;
            
env.close()
show_video()

# REINFORCE Algorithm Experimentation

In [99]:
#@title Imports
from importlib import reload
import gym

import MLP_policy
reload(MLP_policy)
from MLP_policy import MLPPolicy

import utils
reload(utils)

<module 'utils' from '/content/gdrive/My Drive/DeepRL_Experimentation/utils.py'>

In [100]:
env = gym.make("CartPole-v1")

ac_dim = env.action_space.n
ob_dim = env.observation_space.shape[0]
n_layers = 5
hidden_layer_size = 100
learning_rate = 1e-4

policy = MLPPolicy(ac_dim, ob_dim, n_layers, hidden_layer_size, learning_rate)

print(f"E[rewards_sum] before training: {utils.expected_rewards_sum(env, policy, env._max_episode_steps, 200)}")

# Training Loop
for _ in range(500):
  trajectories = utils.sample_n_trajectories(env, policy, env._max_episode_steps, 20)
  policy.update(trajectories)

print(f"E[rewards_sum] after training: {utils.expected_rewards_sum(env, policy, env._max_episode_steps, 200)}")

E[rewards_sum] before training: 22.59
E[rewards_sum] after training: 455.315


In [105]:
from colab_utils import (
    wrap_env,
    show_video
)

env = wrap_env(env)

ob = env.reset()
for i in range(500):
    env.render(mode='rgb_array')
    ob, reward, done, _ = env.step(policy.get_action(ob)) 
    if done:
      break;
            
env.close()
show_video()